In [ ]:
import pyecap
import os
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.io as sio
from scipy.signal import find_peaks
from fnirs_functions import *

mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['axes.linewidth'] = 1

In [ ]:
meta_index = 9

metaDF = pd.read_excel(r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\metadata.xlsx')
tank = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_index, 'Tank']
flex_folder = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_index, 'fNIRs Run'] + '.mat'

#metaDF = pd.read_excel(r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\metadata.xlsx')
#tank = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_index, 'Tank']
#flex_folder = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_index, 'fNIRs Run'] + '.mat'

fd = flexNIRs(flex_folder)

ecg_data = pyecap.Ephys(tank, stores = 'ECGG')
ecg_data = ecg_data.remove_ch(channels=['ECGG 2','ECGG 3', 'ECGG 4'])

stim = pyecap.Stim(tank)
stimDF = stim.parameters

In [ ]:
"""Blign fNIRs Brtifact"""
alignDF = stimDF.loc[stimDF['pulse amplitude (μA)'] < 0]
fd.manual_alignment(stimDF = alignDF, stim_start_index = int(80.5*266.6))
fd.plot_artifact(channel = 'D1 Ambient', fc = 1, filter_type = 'highpass', show_stim = True)

In [ ]:
"""Performs short-channel regression to generate SSR-DF"""
data_type = 'Hemo_filt' # Data type of original data to run SSR regression on
fd.ssr_regression(data_type = data_type)
fd.ssr_plot(data_type, channel = 'D1 LL',wv = 'HbT', show_stim=True, show_ss=True)

In [ ]:
"""GigaPlot 1 - Single Channel"""
fig = plt.figure(layout='constrained', figsize=(15,6))
d_type = 'SSR_filt'
chan = 'D1 LL'
color = 'HbT'
z_shift = False
pre_time = 0
post_time = 0
meta_params = str(metaDF.loc[meta_index, 'Amplitude (mA)']) + ' mA, ' + str(metaDF.loc[meta_index, 'Frequency']) + ' Hz'

mosaic = [['full','full', 'full', 'full', 'full', 'full'],
      ['avg', 'channel stack','avg SS', 'SS stack', 'avg interstim', 'interstim stack']]

ax_dict = fig.subplot_mosaic(mosaic)

fd.plot_channel(data_type =d_type, channel  = chan, wv = color, plot_style = 'full', fig_size = (12,6), show = False, axis = ax_dict['full'])

ax2 = ax_dict['full'].twinx()
fd.plot_channel(data_type =fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'full', fig_size = (12,6), show = False, axis = ax2, color = 'red')
ax2.legend(loc = 'lower right')

# Plot mean & stack of channel
fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['avg'])
fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['channel stack'])

# Plot mean & stack of short-channel

fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['avg SS'])
fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['SS stack'])

# Plot mean & stack of interstim time
fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'interstim mean', zero_shift = z_shift, show = False, axis = ax_dict['avg interstim'])
fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'interstim stacked', zero_shift = z_shift, show = False, axis = ax_dict['interstim stack'])
fig.suptitle(chan + ' ' + color + ': ' + meta_params)
plt.show()

In [ ]:
"""GigaPlot 1 --  Loop"""
d_type = 'SSR_filt'
channels = ['D1 LL', 'D1 LS', 'D3 LL', 'D3 LS']
color = 'HbT'
z_shift = False
pre_time = 0
post_time = 0

mosaic = [['c0_full','c0_full', 'c0_full', 'c0_full', 'c0_full', 'c0_full'],
      ['c0 avg', 'c0 stack','c0 avgSS', 'c0 stackSS', 'c0 avg interstim', 'c0 interstim stack'],
          ['c1_full','c1_full', 'c1_full', 'c1_full', 'c1_full', 'c1_full'],
      ['c1 avg', 'c1 stack','c1 avgSS', 'c1 stackSS', 'c1 avg interstim', 'c1 interstim stack'],
          ['c2_full','c2_full', 'c2_full', 'c2_full', 'c2_full', 'c2_full'],
      ['c2 avg', 'c2 stack','c2 avgSS', 'c2 stackSS', 'c2 avg interstim', 'c2 interstim stack'],
          ['c3_full','c3_full', 'c3_full', 'c3_full', 'c3_full', 'c3_full'],
      ['c3 avg', 'c3 stack','c3 avgSS', 'c3 stackSS', 'c3 avg interstim', 'c3 interstim stack'],]

metaLIST = [2,3,4,5,7,9,10,11,12,13]
#metaLIST = [2]

for idx, meta_idx in enumerate(metaLIST):
    tank = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_idx, 'Tank']
    flex_folder = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_idx, 'fNIRs Run'] + '.mat'

    #metaDF = pd.read_excel(r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\metadata.xlsx')
    #tank = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_index, 'Tank']
    #flex_folder = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_index, 'fNIRs Run'] + '.mat'

    fd = flexNIRs(flex_folder)
    stim = pyecap.Stim(tank)
    stimDF = stim.parameters

    ecg_data = pyecap.Ephys(tank, stores = 'ECGG')
    ecg_data = ecg_data.remove_ch(channels=['ECGG 2','ECGG 3', 'ECGG 4'])

    alignDF = stimDF.loc[stimDF['pulse amplitude (μA)'] < 0]
    fd.manual_alignment(stimDF = alignDF, stim_start_index = int(metaDF.loc[meta_idx, 'fNIRs stim start time']*266.6))
    fd.ssr_regression(data_type = 'Hemo_filt')

    fig = plt.figure(figsize=(16,24), layout='constrained')
    ax_dict = fig.subplot_mosaic(mosaic, gridspec_kw={'wspace':0, 'hspace':0})
    fig.get_layout_engine().set(w_pad = -10, hspace = -10)

    meta_params = str(metaDF.loc[meta_idx, 'Amplitude (mA)']) + ' mA, ' + str(metaDF.loc[meta_idx, 'Frequency']) + ' Hz'
    save_title = str(metaDF.loc[meta_idx, 'Amplitude (mA)']) + '_mA_' + str(metaDF.loc[meta_idx, 'Frequency']) + '_Hz'

    for idx, chan in enumerate(channels):

        fd.plot_channel(data_type =d_type, channel  = chan, wv = color, plot_style = 'full', fig_size = (12,6), show = False, axis = ax_dict['c' + str(idx) + '_full'])
        ax_dict['c' + str(idx) + '_full'].set(title = chan)
        #Adds SS plot
        ax2 = ax_dict['c' + str(idx) + '_full'].twinx()
        fd.plot_channel(data_type =fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'full', fig_size = (12,6), show = False, axis = ax2, color = 'red')
        ax2.legend(loc = 'lower right')

        # Plot mean & stack of channel
        fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['c' + str(idx) + ' avg'])
        ax_dict['c' + str(idx) + ' avg'].set(ylabel=None)
        fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['c' + str(idx) + ' stack'])
        ax_dict['c' + str(idx) + ' stack'].set(ylabel=None)


        # Plot mean & stack of short-channel

        fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['c' + str(idx) + ' avgSS'])
        ax_dict['c' + str(idx) + ' avgSS'].set(ylabel=None)
        fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['c' + str(idx) + ' stackSS'])
        ax_dict['c' + str(idx) + ' stackSS'].set(ylabel=None)

        # Plot mean & stack of interstim time
        fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'interstim mean', zero_shift = z_shift, show = False, axis = ax_dict['c' + str(idx) + ' avg interstim'])
        ax_dict['c' + str(idx) + ' avg interstim'].set(ylabel=None)
        fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'interstim stacked', zero_shift = z_shift, show = False, axis = ax_dict['c' + str(idx) + ' interstim stack'])
        ax_dict['c' + str(idx) + ' interstim stack'].set(ylabel=None)

    fig.suptitle(meta_params)
    fig.savefig('KL_' + save_title + '.png', bbox_inches='tight',format='png')
    plt.show()

In [ ]:
 """GigaPlot 2 -- Single Channel
Show mean/individual stims for a channel at increasing frequency across a row, each row is a different amplitude
how to work in SS channel?  Each row is a set of pairs with increasing frequency?  6 plots per row
"""
metaLIST3 = [5,2,3,4] #3 mA
metaLIST4 = [7,9] #4 mA
metaLIST6 = [12,11,13,10] #6 mA

freqLIST = [17,100,200,300] #3 mA

fig = plt.figure(layout='constrained', figsize=(15,8.5))
d_type = 'SSR_filt'
chan = 'D1 LL'
color = 'HbT'
z_shift = False
pre_time = 0
post_time = 0
#meta_params = str(metaDF.loc[meta_index, 'Amplitude (mB)']) + ' mB, ' + str(metaDF.loc[meta_index, 'Frequency']) + ' Hz'

mosaic = [['A_F0_ch','A_F0_ss', 'A_F1_ch', 'A_F1_ss', 'A_F2_ch', 'A_F2_ss','A_F3_ch', 'A_F3_ss'],
          ['B_F0_ch','B_F0_ss', 'B_F1_ch', 'B_F1_ss', 'B_F2_ch', 'B_F2_ss','B_F3_ch', 'B_F3_ss'],
          ['C_F0_ch','C_F0_ss', 'C_F1_ch', 'C_F1_ss', 'C_F2_ch', 'C_F2_ss','C_F3_ch', 'C_F3_ss'],]

ax_dict = fig.subplot_mosaic(mosaic)

for idx, meta_idx in enumerate(metaLIST3):
    tank = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_idx, 'Tank']
    flex_folder = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_idx, 'fNIRs Run'] + '.mat'

    #metaDF = pd.read_excel(r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\metadata.xlsx')
    #tank = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_index, 'Tank']
    #flex_folder = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_index, 'fNIRs Run'] + '.mat'

    fd = flexNIRs(flex_folder)
    stim = pyecap.Stim(tank)
    stimDF = stim.parameters

    ecg_data = pyecap.Ephys(tank, stores = 'ECGG')
    ecg_data = ecg_data.remove_ch(channels=['ECGG 2','ECGG 3', 'ECGG 4'])

    alignDF = stimDF.loc[stimDF['pulse amplitude (μA)'] < 0]
    fd.manual_alignment(stimDF = alignDF, stim_start_index = int(60.5*266.6))
    fd.ssr_regression(data_type = 'Hemo_filt')

    # Plot mean & stack of channel
    fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['A_F' + str(idx) + '_ch'])
    # fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['channel stack'])

    ax_dict['A_F' + str(idx) + '_ch'].set(title = str(freqLIST[idx]) + ' Hz')#, ' + chan)
    #ax_dict['A_F' + str(idx) + '_ch'].xaxis.grid(True)

    # Plot mean & stack of short-channel

    fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['A_F' + str(idx) + '_ss'])
    ax_dict['A_F' + str(idx) + '_ss'].set(title = str(freqLIST[idx]) + ' Hz, SS')
    # fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['SS stack'])

for idx, meta_idx in enumerate(metaLIST4):
    tank = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_idx, 'Tank']
    flex_folder = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_idx, 'fNIRs Run'] + '.mat'

    #metaDF = pd.read_excel(r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\metadata.xlsx')
    #tank = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_index, 'Tank']
    #flex_folder = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_index, 'fNIRs Run'] + '.mat'

    fd = flexNIRs(flex_folder)
    stim = pyecap.Stim(tank)
    stimDF = stim.parameters

    ecg_data = pyecap.Ephys(tank, stores = 'ECGG')
    ecg_data = ecg_data.remove_ch(channels=['ECGG 2','ECGG 3', 'ECGG 4'])

    alignDF = stimDF.loc[stimDF['pulse amplitude (μA)'] < 0]
    if idx == 1:
        fd.manual_alignment(stimDF = alignDF, stim_start_index = int(80.5*266.6))
    else:
        fd.manual_alignment(stimDF = alignDF, stim_start_index = int(60.5*266.6))
    fd.ssr_regression(data_type = 'Hemo_filt')

    # Plot mean & stack of channel
    fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['B_F' + str(idx+2) + '_ch'])
    # fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['channel stack'])

    ax_dict['B_F' + str(idx+2) + '_ch'].set(title = str(freqLIST[idx+2]) + ' Hz')#, ' + chan)

    # Plot mean & stack of short-channel

    fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['B_F' + str(idx+2) + '_ss'])
    ax_dict['B_F' + str(idx+2) + '_ss'].set(title = str(freqLIST[idx+2]) + ' Hz, SS')
    # fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['SS stack'])

for idx, meta_idx in enumerate(metaLIST6):
    tank = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_idx, 'Tank']
    flex_folder = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_idx, 'fNIRs Run'] + '.mat'

    #metaDF = pd.read_excel(r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\metadata.xlsx')
    #tank = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_index, 'Tank']
    #flex_folder = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_index, 'fNIRs Run'] + '.mat'

    fd = flexNIRs(flex_folder)
    stim = pyecap.Stim(tank)
    stimDF = stim.parameters

    ecg_data = pyecap.Ephys(tank, stores = 'ECGG')
    ecg_data = ecg_data.remove_ch(channels=['ECGG 2','ECGG 3', 'ECGG 4'])

    alignDF = stimDF.loc[stimDF['pulse amplitude (μA)'] < 0]
    fd.manual_alignment(stimDF = alignDF, stim_start_index = int(60.5*266.6))
    fd.ssr_regression(data_type = 'Hemo_filt')

    # Plot mean & stack of channel
    fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['C_F' + str(idx) + '_ch'])
    # fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['channel stack'])

    ax_dict['C_F' + str(idx) + '_ch'].set(title = str(freqLIST[idx]) + ' Hz')#, ' + chan)

    # Plot mean & stack of short-channel

    fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['C_F' + str(idx) + '_ss'])
    ax_dict['C_F' + str(idx) + '_ss'].set(title = str(freqLIST[idx]) + ' Hz, SS')
    # fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['SS stack'])

ax_dict['A_F0_ch'].set(ylabel = '3 mA')
ax_dict['B_F0_ch'].set(ylabel = '4 mA')
ax_dict['C_F0_ch'].set(ylabel = '6 mA')

for ax in ax_dict:
    for x in [2,4,6,8]:
        ax_dict[ax].axvline(x, color = 'black', alpha = 0.2)

plt.suptitle(chan + ' HbT with SSR regression')
plt.show()

In [ ]:
"""GigaPlot 2 -- Loop
Show mean/individual stims for a channel at increasing frequency across a row, each row is a different amplitude
how to work in SS channel?  Each row is a set of pairs with increasing frequency?  6 plots per row
"""
metaLIST3 = [5,2,3,4] #3 mA
metaLIST4 = [7,9] #4 mA
metaLIST6 = [12,11,13,10] #6 mA

freqLIST = [17,100,200,300] #3 mA


d_type = 'SSR_filt'
channels = ['D1 LL', 'D1 LS', 'D3 LL', 'D3 LS']
color = 'HbT'
z_shift = False
pre_time = 0
post_time = 0
#meta_params = str(metaDF.loc[meta_index, 'Amplitude (mB)']) + ' mB, ' + str(metaDF.loc[meta_index, 'Frequency']) + ' Hz'

mosaic = [['A_F0_ch','A_F0_ss', 'A_F1_ch', 'A_F1_ss', 'A_F2_ch', 'A_F2_ss','A_F3_ch', 'A_F3_ss'],
          ['B_F0_ch','B_F0_ss', 'B_F1_ch', 'B_F1_ss', 'B_F2_ch', 'B_F2_ss','B_F3_ch', 'B_F3_ss'],
          ['C_F0_ch','C_F0_ss', 'C_F1_ch', 'C_F1_ss', 'C_F2_ch', 'C_F2_ss','C_F3_ch', 'C_F3_ss'],]
for chan in channels:
    fig = plt.figure(layout='constrained', figsize=(15,8.5))
    ax_dict = fig.subplot_mosaic(mosaic)

    for idx, meta_idx in enumerate(metaLIST3):
        tank = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_idx, 'Tank']
        flex_folder = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_idx, 'fNIRs Run'] + '.mat'

        #metaDF = pd.read_excel(r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\metadata.xlsx')
        #tank = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_index, 'Tank']
        #flex_folder = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_index, 'fNIRs Run'] + '.mat'

        fd = flexNIRs(flex_folder)
        stim = pyecap.Stim(tank)
        stimDF = stim.parameters

        ecg_data = pyecap.Ephys(tank, stores = 'ECGG')
        ecg_data = ecg_data.remove_ch(channels=['ECGG 2','ECGG 3', 'ECGG 4'])

        alignDF = stimDF.loc[stimDF['pulse amplitude (μA)'] < 0]
        fd.manual_alignment(stimDF = alignDF, stim_start_index = int(60.5*266.6))
        fd.ssr_regression(data_type = 'Hemo_filt')

        # Plot mean & stack of channel
        fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['A_F' + str(idx) + '_ch'])
        # fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['channel stack'])

        ax_dict['A_F' + str(idx) + '_ch'].set(title = str(freqLIST[idx]) + ' Hz')#, ' + chan)
        #ax_dict['A_F' + str(idx) + '_ch'].xaxis.grid(True)

        # Plot mean & stack of short-channel

        fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['A_F' + str(idx) + '_ss'])
        ax_dict['A_F' + str(idx) + '_ss'].set(title = str(freqLIST[idx]) + ' Hz, SS')
        # fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['SS stack'])

    for idx, meta_idx in enumerate(metaLIST4):
        tank = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_idx, 'Tank']
        flex_folder = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_idx, 'fNIRs Run'] + '.mat'

        #metaDF = pd.read_excel(r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\metadata.xlsx')
        #tank = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_index, 'Tank']
        #flex_folder = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_index, 'fNIRs Run'] + '.mat'

        fd = flexNIRs(flex_folder)
        stim = pyecap.Stim(tank)
        stimDF = stim.parameters

        ecg_data = pyecap.Ephys(tank, stores = 'ECGG')
        ecg_data = ecg_data.remove_ch(channels=['ECGG 2','ECGG 3', 'ECGG 4'])

        alignDF = stimDF.loc[stimDF['pulse amplitude (μA)'] < 0]
        if idx == 1:
            fd.manual_alignment(stimDF = alignDF, stim_start_index = int(80.5*266.6))
        else:
            fd.manual_alignment(stimDF = alignDF, stim_start_index = int(60.5*266.6))
        fd.ssr_regression(data_type = 'Hemo_filt')

        # Plot mean & stack of channel
        fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['B_F' + str(idx+2) + '_ch'])
        # fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['channel stack'])

        ax_dict['B_F' + str(idx+2) + '_ch'].set(title = str(freqLIST[idx+2]) + ' Hz')#, ' + chan)

        # Plot mean & stack of short-channel

        fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['B_F' + str(idx+2) + '_ss'])
        ax_dict['B_F' + str(idx+2) + '_ss'].set(title = str(freqLIST[idx+2]) + ' Hz, SS')
        # fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['SS stack'])

    for idx, meta_idx in enumerate(metaLIST6):
        tank = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_idx, 'Tank']
        flex_folder = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_idx, 'fNIRs Run'] + '.mat'

        #metaDF = pd.read_excel(r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\metadata.xlsx')
        #tank = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_index, 'Tank']
        #flex_folder = r'D:\Data\TDT_fNIRs\20260213_fNIRs_QC\flexNIRs\\' + metaDF.loc[meta_index, 'fNIRs Run'] + '.mat'

        fd = flexNIRs(flex_folder)
        stim = pyecap.Stim(tank)
        stimDF = stim.parameters

        ecg_data = pyecap.Ephys(tank, stores = 'ECGG')
        ecg_data = ecg_data.remove_ch(channels=['ECGG 2','ECGG 3', 'ECGG 4'])

        alignDF = stimDF.loc[stimDF['pulse amplitude (μA)'] < 0]
        fd.manual_alignment(stimDF = alignDF, stim_start_index = int(60.5*266.6))
        fd.ssr_regression(data_type = 'Hemo_filt')

        # Plot mean & stack of channel
        fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['C_F' + str(idx) + '_ch'])
        # fd.plot_channel(data_type = d_type, channel  = chan, wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['channel stack'])

        ax_dict['C_F' + str(idx) + '_ch'].set(title = str(freqLIST[idx]) + ' Hz')#, ' + chan)

        # Plot mean & stack of short-channel

        fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'mean', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['C_F' + str(idx) + '_ss'])
        ax_dict['C_F' + str(idx) + '_ss'].set(title = str(freqLIST[idx]) + ' Hz, SS')
        # fd.plot_channel(data_type = fd.SSR_dtype, channel  = 'SS', wv = color, plot_style = 'stacked', pre_time = pre_time, post_time = post_time, fig_size = (12,6), zero_shift = z_shift, show = False, axis = ax_dict['SS stack'])

    ax_dict['A_F0_ch'].set(ylabel = '3 mA')
    ax_dict['B_F0_ch'].set(ylabel = '4 mA')
    ax_dict['C_F0_ch'].set(ylabel = '6 mA')

    for ax in ax_dict:
        for x in [2,4,6,8]:
            ax_dict[ax].axvline(x, color = 'black', alpha = 0.2)

    plt.suptitle(chan + ' HbT with SSR regression')
    plt.savefig('KL_' + chan + '_HbT_gigaplot2.png', bbox_inches = 'tight', format = 'png')
    plt.show()

In [ ]:
"""Plot for data presentation
Plots of interest:
HbO/HbR or each channel with different stims
Priority: HbT of each channel of different stim parameters overlaid on top of each to assess differences in stim amplitude effects?
Plot showing effect of SSR regression data
Bdd HR as a secondary plot?

Bdd averages of inter-stim "off" periods between to gigaplots

gigaplot #1
Top most trace = Full recording (with or without SS data?)
Middle Plot = ECG?
Bottom rows of plots
Row #1
a: average of stim
b: stacked stim traces
c: average of SS channel
d: stacked SS channel
Row #2
a: average of baseline following interested channel
b: stacked during baseline
c: average of ss channel during baseline
d: stacked SS during baseline

gigaplot #2 - side-by-side stims
Single row of plots where each plot is a stimulation with the original data, ssr data, and SS data
"""

In [ ]:
plotDF = fd.get_data(data_type = 'Mua_filt')
stimDF = fd.stimDF

onset_indices = stimDF['fNIRs onset index'].values
offset_indices = stimDF['fNIRs offset index'].values

interstim_idx = [ (offset_indices[i], onset_indices[i+1]) for i in np.arange(len(onset_indices)-1)]
interstim_sample_len = [idx2 - idx1 for idx1,idx2 in interstim_idx]
interstim_idx.append( (offset_indices[-1], offset_indices[-1] + interstim_sample_len[0] ))
# for idx, stim in enumerate(stimDF.index):
#
#     #Start case
#     if idx == 0:
#         idx1 = stimDF.loc[stim, 'fNIRs offset index']
#         idx2 = stimDF.loc[stim, 'fNIRs onset index']
#
#     print(stim)

    #
    # stim_start_idx = stimDF.loc[stim, 'fNIRs onset index']
    # stim_stop_idx = stimDF.loc[stim, 'fNIRs offset index']
    #
    # plotDF.loc[plot_start_idx:plot_stop_idx, 'Stim #'] = str(idx + 1)
    #
    # # Construct time array based on sampling frequency and number of indices with time zero at stimulation start
    # time_span = plot_stop_idx - plot_start_idx + 1
    # time = np.arange(-pre_BL_idx_width, time_span - pre_BL_idx_width, 1) / fs
    #
    # plotDF.loc[plot_start_idx:plot_stop_idx, 'Trial Time'] = time
    #

    #End case

In [ ]:
interstim_sample_len = [idx2 - idx1 for idx1,idx2 in interstim_idx]
interstim_idx.append( (offset_indices[-1], offset_indices[-1] + interstim_sample_len[0] ))

In [ ]:
onset_indices = stimDF['fNIRs onset index'].values
offset_indices = stimDF['fNIRs offset index'].values

interstim_idx = [(offset_indices[i], onset_indices[i + 1]) for i in np.arange(len(onset_indices) - 1)]
interstim_sample_len = [idx2 - idx1 for idx1, idx2 in interstim_idx]
interstim_idx.append((offset_indices[-1], offset_indices[-1] + interstim_sample_len[-1]))

for idx, (start, stop) in enumerate(interstim_idx):
    plotDF.loc[start:stop, 'Interstim #'] = str(idx + 1)

    # Construct time array based on sampling frequency and number of indices with time zero at stimulation start
    time_span = stop - start + 1
    time = np.arange(0, time_span, 1) / fd.fs

    plotDF.loc[start:stop, 'Trial Time'] = time

In [ ]:
data_type = 'SSR_filt'
channels = ['D1 LS', 'D1 LL', 'D3 LS', 'D3 LL']

for chan in channels:
    fd.gigaplot(data_type, channel = chan, pre_time=5, post_time=5,show_stim=True, fig_size = (12,6), zero_shift = True)

In [ ]:
channels = ['D1 LL', 'D1 LS']
color_chan = ['Red','IR']
chs = [ch + ' ' + color for color in color_chan for ch in channels]
chs

In [ ]:
channel = 'D1 LL'
channel_pair = [ fd.ch_dct[ch][0] + ' ' + fd.ch_dct[ch][1] + ' SSR' for ch in fd.ch_pairs[channel]]

channel_pair = [ch.replace('Red', 'HbO') for ch in channel_pair]
channel_pair = [ch.replace('IR', 'HbR') for ch in channel_pair]

print(channel_pair)

In [ ]:
fd.plot_channel(data_type = 'SSR_filt', channel = 'D3 LL', pre_time =5, post_time = 5, plot_style='Full', fig_size =(4,10), zero_shift = True)

In [ ]:
"""Perform HR calculations"""
#fNIRs HR
fd.calc_hr(channel = 'SS Red', filter_cutoffs=(0.5,15),peak_height = 2000, check_plot = False)

#Filter TDT ECG without Down-sampling
tdt_ecg = filter_hr(ecg_data, cutoff = 10, downsample=False, check_plot=False)
ecgDF = calc_hr(tdt_ecg, peak_height = 250, fs = ecg_data.sample_rate)

#Filter and down sample TDT ECG data for Plotting
DS_factor = 10
tdt_ecg_DS = filter_hr(ecg_data, cutoff = 10, downsample=True, downsample_factor = DS_factor, check_plot=False)
ecgDF_DS = calc_hr(tdt_ecg_DS, peak_height = 250, fs = ecg_data.sample_rate / DS_factor)

In [ ]:
"""Plot ECG"""
fig = go.Figure()
fig.add_trace(go.Scatter(x = ecgDF['Time (s)'], y = ecgDF['smooth bpm'], name = 'TDT HR'))
#fig.add_trace(go.Scatter(x = fd.ecgDF['Time (s)'], y = fd.ecgDF['b2b bpm'],name = 'fNIRs HR'))
#fig.add_trace(go.Scatter(x = ecgDF_DS['Time (s)'], y = ecgDF_DS['b2b bpm'], name = 'downsampled'))
for param in stimDF.index:
        fig.add_vrect(x0=fd.stimDF.loc[param]['onset time (s)'],
                  x1=fd.stimDF.loc[param]['offset time (s)'],
                      line_width=0, fillcolor='red', opacity=0.15)
fig.show()

In [ ]:
fd.plot_channel_interactive(data_type = 'SSR_filt', channel = 'D1 LL Red SSR', show_stim =True, show_hr=True, hr_chan='smooth')

In [ ]:
data_type = 'Mua_filt' # Data type of original data
fd.ssr_regression(data_type = data_type)
#fd.ssr_plot(data_type, channel = 'D1 LL Red', show_stim=True, show_ss=True)

In [ ]:
"""Plot for data presentation
Plots of interest:
HbO/HbR or each channel with different stims
Priority: HbT of each channel of different stim parameters overlaid on top of each to assess differences in stim amplitude effects?
Plot showing effect of SSR regression data
Bdd HR as a secondary plot?
"""
fd.plot_channel(data_type = 'SSR_filt', channel = 'D1 LL', pre_time =5, post_time = 5, plot_style='Full', fig_size =(4,10), zero_shift = True)

In [ ]:
"""Stacked ECG plot"""
#metaLIST = [5,2,3,4] #3 mB
#metaLIST = [6,8,7,9] #4 mB
metaLIST = [12,11,13,10] #6 mB

freqLIST = [17, 100,200,300] #3 mB

fig,ax = plt.subplots(nrows = 4, figsize = (10,10), layout='constrained')
ax = ax.ravel()

for idx, meta_index in enumerate(metaLIST):
    tank = r'G:\Data\fNIRs_qc\20260213_fNIRs_QC\CVP_SingleCh_Ramp-260213\\' + metaDF.at[meta_index, 'Tank']
    ecg_data = pyecap.Ephys(tank, stores = 'ECGG')
    ecg_data = ecg_data.remove_ch(channels=['ECGG 2','ECGG 3', 'ECGG 4'])
    stim = pyecap.Stim(tank)
    stimDF = stim.parameters

    #Filter TDT ECG without Down-sampling
    tdt_ecg = filter_hr(ecg_data, cutoff = 10, downsample=False, check_plot=False)
    ecgDF = calc_hr(tdt_ecg, peak_height = 250, fs = ecg_data.sample_rate)

    title = '6 mB, ' + str(freqLIST[idx]) + ' Hz'
    ax[idx].set_xlim( stimDF.loc[stimDF.index[0], 'onset time (s)'] - 60, stimDF.loc[stimDF.index[-1], 'offset time (s)'] + 60 )
    sns.lineplot(data = ecgDF, x = ecgDF['Time (s)'], y = ecgDF['smooth bpm'], ax=ax[idx], label = 'HR (bpm)').set_title(title)
    ax[idx].set_ylim(50,70)

    for param in stimDF.index:
        ax[idx].axvspan(xmin=stimDF.loc[param]['onset time (s)'],
                                            xmax=stimDF.loc[param]['offset time (s)'], color='red', alpha=0.1)

In [ ]:
fs = fd.fs
pre_time = 5
post_time = 5
zero_shift = True

ss_chan = 'SS HbT'

channels = ['D1 LS', 'D1 LL', 'D3 LS', 'D3 LL']

pre_BL_idx_width = int(pre_time * fs)
post_BL_idx_width = int(post_time * fs)

for chan in channels:

    plotDF = pd.DataFrame(fd.get_data('SSR_filt', chan, wv = 'HbT'))
    plotDF['Time (s)'] = fd.time

    #Bdd SS HbT channel
    ssDF = fd.get_data('Hemo_filt')
    ssDF['Time (s)'] = fd.time

    stimDF = fd.stimDF.copy()
    # Label data points based on stimulation #
    for idx, stim in enumerate(stimDF.index):
        stim_start_idx = stimDF.loc[stim, 'fNIRs onset index']
        stim_stop_idx = stimDF.loc[stim, 'fNIRs offset index']

        plot_start_idx = (stim_start_idx - pre_BL_idx_width).astype(int)
        plot_stop_idx = (stim_stop_idx + post_BL_idx_width).astype(int)

        if plot_stop_idx > len(plotDF):
            raise Exception('Plot stop index exceeds plot length. Reduce post-stimulation time.')

        plotDF.loc[plot_start_idx:plot_stop_idx, 'Stim #'] = str(idx + 1)
        ssDF.loc[plot_start_idx:plot_stop_idx, 'Stim #'] = str(idx + 1)

        # Construct time array based on sampling frequency and number of indices with time zero at stimulation start
        time_span = plot_stop_idx - plot_start_idx + 1
        time = np.arange(-pre_BL_idx_width, time_span - pre_BL_idx_width, 1) / fs

        plotDF.loc[plot_start_idx:plot_stop_idx, 'Trial Time'] = time
        ssDF.loc[plot_start_idx:plot_stop_idx, 'Trial Time'] = time

    #Construct super figure
    fig = plt.figure(layout='constrained', figsize=(9,6))

    # mosaic = [['full', 'full', 'full', 'full'],
    #           ['D1 LS avg', 'D1 LL avg', 'D3 LS avg', 'D3 LL avg'],
    #           ['D1 LS Red', 'D1 LL Red', 'D3 LS Red', 'D3 LL Red'],
    #           ['D1 LS IR', 'D1 LL IR', 'D3 LS IR', 'D3 LL IR'], ]

    mosaic = [['full','full', 'full', 'full'],
          ['avg', 'Red','avg SS', 'SS stack']]

    ax_dict = fig.subplot_mosaic(mosaic)

    #Top Plot = Whole trace with shaded stim boxed
    plot_chan = chan + ' ' + 'HbT'

    sns.lineplot(data = plotDF, x = plotDF['Time (s)'], y = plotDF[plot_chan], ax = ax_dict['full'], label = plot_chan)
    #sns.lineplot(data = ssDF, x = ssDF['Time (s)'], y = ssDF[ss_chan], ax = ax_dict['full'], label = 'SS HbT')
    for param in stimDF.index:
        ax_dict['full'].axvspan(xmin=stimDF.loc[param]['fNIRs onset time (s)'],
                                xmax=stimDF.loc[param]['fNIRs offset time (s)'], color='red', alpha=0.05)

    plotDF.dropna(axis=0, subset=['Stim #'], inplace=True)
    ssDF.dropna(axis=0, subset=['Stim #'], inplace=True)

    # Bvg. Response during stim Plot
    if zero_shift:
        for stim_num in plotDF['Stim #'].unique():
            zero_point = plotDF.loc[plotDF['Stim #'] == stim_num, 'Trial Time'].abs().idxmin()
            plotDF.loc[plotDF['Stim #'] == stim_num, plot_chan] = (
                    plotDF.loc[plotDF['Stim #'] == stim_num, plot_chan] - plotDF.loc[
                zero_point, plot_chan].item())

            ss_zero_point = ssDF.loc[ssDF['Stim #'] == stim_num, 'Trial Time'].abs().idxmin()
            ssDF.loc[ssDF['Stim #'] == stim_num, ss_chan] = (
                    ssDF.loc[ssDF['Stim #'] == stim_num, ss_chan] - ssDF.loc[
                ss_zero_point, ss_chan].item())


    sns.lineplot(data=plotDF, x='Trial Time', y=plot_chan, errorbar='sd', ax=ax_dict['avg'], label = 'HbT')
    sns.lineplot(data=plotDF, x='Trial Time', y=plot_chan, hue='Stim #', ax=ax_dict['Red'], legend = False)

    sns.lineplot(data=ssDF, x='Trial Time', y=ss_chan, errorbar='sd', ax=ax_dict['avg SS'], label = 'SS HbT')
    sns.lineplot(data=ssDF, x='Trial Time', y=ss_chan, hue = 'Stim #', ax=ax_dict['SS stack'], legend = False)

    ax_dict['avg'].axvspan(xmin=0, xmax=stimDF.loc[stim, 'duration (ms)'] * 1e-3, color='red', alpha=0.05)
    ax_dict['Red'].axvspan(xmin=0, xmax=stimDF.loc[stim, 'duration (ms)'] * 1e-3, color='red', alpha=0.05)
    ax_dict['avg SS'].axvspan(xmin=0, xmax=stimDF.loc[stim, 'duration (ms)'] * 1e-3, color='red', alpha=0.05)
    ax_dict['SS stack'].axvspan(xmin=0, xmax=stimDF.loc[stim, 'duration (ms)'] * 1e-3, color='red', alpha=0.05)

In [ ]:
|